## Таблиця населених пунктів України - назви, коди КОАТУУ, населення, координати

Почищена, виправлена, з доданням кодів КОАТУУ та збереженням старих назв (поруч із декомунізованими). Є усі села і найменші селища України (звісно, 

In [1]:
import pandas as pd
import numpy as np
import re

Крок 1: дістати назви та КОАТУУ-коди областей.

In [2]:
wikiurl = 'https://uk.wikipedia.org/wiki/%D0%A1%D0%BF%D0%B8%D1%81%D0%BE%D0%BA_%D0%BA%D0%BE%D0%B4%D1%96%D0%B2_%D0%9A%D0%9E%D0%90%D0%A2%D0%A3%D0%A3_%D0%B4%D0%BB%D1%8F_%D0%BE%D0%B1%D0%BB%D0%B0%D1%81%D1%82%D0%B5%D0%B9'
oblcodes = pd.read_html(wikiurl, skiprows=2, header=None)[0]
oblcodes = oblcodes.iloc[:,1:]
oblcodes.columns = ['oblcode', 'obl']

oblcodes['obl'] = oblcodes['obl'].apply(lambda x: x.split('/')[0]).str.strip()
oblcodes['oblcode'] = oblcodes['oblcode'].astype(str).apply(
    lambda x: ('0'+x) if (len(x) < 2) else x)

Крок 2: відкрити класифікатор КОАТУУ від Держстату та розібратись із ним. Додати коди областей.

Колонка "np" вказує на тип поселення, включаючи М (міста), Р (райони міст) , Т (смт), С (села), Щ (селища - маленькі села, грубо кажучи). Я потім додав також В (великі міста, які діляться на окремі райони і не мали типу).

In [3]:
# current KOATUU is from 26-12-2016
koatuu = pd.read_excel('KOATUU_26122016.xls', sheetname=0)
koatuu.columns = ['koatuu', 'np', 'town']

def fix_code(cell):
    if len(str(cell)) == 9:
        return '0'+str(cell)
    else:
        return str(cell)

def remove_caps(cell):
    pos = cell.find("’")
    if pos != -1:
        newcell = (cell[:pos+1]) + (cell[pos+1]).lower() + (cell[pos+2:])
        return newcell
    else:
        return cell

koatuu['koatuu'] = koatuu['koatuu'].apply(fix_code)
koatuu['oblcode'] = koatuu['koatuu'].apply(lambda x: x[:2])
koatuu['town'] = koatuu['town'].str.replace("'", "’")
koatuu['np'] = koatuu['np'].str.replace("C", "С")
koatuu = koatuu.merge(oblcodes, on='oblcode', how='left')

Дістаємо назви районів або міст обласного значення, а також робимо колонку "type" із більш адекватними назвами типів.

In [4]:
def get_dis(row):
    code = row['koatuu']
    name = row['town']
    if re.match(r'\d[1-9][1-9]\d{2}00000', code):
        if not re.match(r'\d{3}0000000', code):
            return name
    elif re.match(r'80[1-9][1-9]\d00000', code):
        return name
    elif (code == '8000000000') or (code == '8500000000'):
        return name
    return np.nan

def get_big_type(row):
    code = row['koatuu']
    type_ = row['np']
    name = row['town']
    if re.match(r'\d[1-9][1-9]\d{2}00000', code):
        if not re.match(r'\d{3}0000000', code):
            if type_ == 'М':
                return 'town'
            elif type_ == 'Р':
                return 'city_dis'
            elif re.match(r'.*РАЙОН.*', name):
                return np.nan
            else:
                return 'city'
    else:
        if type_ == 'С':
            return 'village'
        elif type_ == 'М':
            return 'town'
        elif type_ == 'Т':
            return 'smt'
        elif type_ == 'Щ':
            return 'small_village'
        elif type_ == 'Р':
            return 'city_dis'
        elif (code == '8000000000') or (code == '8500000000'):
            return 'city'
    return np.nan

In [5]:
koatuu['dis'] = koatuu.apply(get_dis, axis=1)
koatuu['type'] = koatuu.apply(get_big_type, axis=1)
koatuu['dis'] = koatuu['dis'].fillna(method='ffill')

In [6]:
koatuu = koatuu.dropna(subset=['type'])
koatuu['dis'] = koatuu['dis'].apply(
    lambda x: x.split('/')[0] if ('/' in x) else x)
koatuu['dis'] = koatuu['dis'].str.title().apply(remove_caps).str.replace('Район', 'район')
koatuu['dis'] = koatuu['dis'].str.replace('М\.', '')

Для виправлення помилок та отримання старих назв беремо таблицю декомунізованих назв від Держстату.

In [7]:
dekom = pd.read_excel('KOATUU_DEKOM_26.12.2016.xls')
dekom.columns = ['koatuu','np','oldname','town','postanova']
dekom = dekom[['koatuu', 'np', 'oldname', 'town']]
dekom['oldname'] = dekom['oldname'].str.replace("'", "’")
dekom['town'] = dekom['town'].str.replace("'", "’")
dekom['koatuu'] = dekom['koatuu'].apply(fix_code)
dekom['oblcode'] = dekom['koatuu'].apply(lambda x: x[:2])

In [8]:
koatuu = koatuu.merge(dekom, on = ['koatuu', 'town', 'oblcode', 'np'], how='left')
koatuu['np'] = koatuu['np'].fillna('В')

In [9]:
koatuu_caps = koatuu.copy()

In [10]:
def get_abbr(row):
    np = row['np']
    town = row['town']
    npdic = {"С": "с. ", "Щ": "с. ", "М": "м. ", "В": "м. ", "Т": "смт ", "Р": "р. "}
    newtown = npdic[np]+remove_caps(town.title())
    return newtown

koatuu['town'] = koatuu.apply(get_abbr, axis=1)
koatuu['town'] = koatuu['town'].str.replace('\sМ\.', '')
koatuu['dis'] = koatuu['dis'].str.replace('М\.', '')
koatuu['town'] = koatuu['town'].str.replace('м\.', 'м. ').str.replace('  ', ' ')

In [11]:
#koatuu.to_csv('koatuu.csv', index=False)

Тепер беремо таблицю зі збірника Держстату "Чисельність наявного населення України на 1 січня 2016 року". Дістаємо області, виправляємо помилки в назвах, дістаємо райони.

In [12]:
pop = pd.read_excel('zb_nas_15xl.xls', 11, skiprows=5)
pop = pop[1:-2]
pop.columns = ['town', '2014', '2015', '2016']
pop = pop.dropna(subset=['town'])

In [13]:
def get_pop_obl(cell):
    if ('Республіка' in cell) or ('область' in cell):
        return cell
    elif 'м. Київ' in cell:
        return 'Київ'
    elif 'Севастополь' in cell:
        return 'Севастополь'
    else:
        return np.nan

pop['town'] = pop['town'].str.replace("'", "’")
pop['obl'] = pop['town'].apply(get_pop_obl)
pop['obl'] = pop['obl'].fillna(method='ffill')

In [14]:
popreplace = {'м. Кіровоград': 'м. Кропивницький',
              'м. Червонопартизанськ': 'м. Вознесенівка',
              'Голуба затока': 'Голуба Затока',
              'Симеїз': 'Сімеїз',
              'Журавне': 'Журавно',
              'Короштишів': 'Коростишів',
              'Великий  Березний': 'Великий Березний',
              'Комінтернівське': 'Доброслав',
              'Сентяківка': 'Сентянівка'}

disreplace = {'Комінтернівський район': 'Лиманський район'}

pop = pop.replace({'town': popreplace}, regex=True)

In [15]:
wrong_towns = ['м. Київ', 'м. Балта', 'м. Біляївка', 'м. Каховка', 'м. Первомайський', 
               'м. Вільногірськ']

In [16]:
def remove_ex(cell):
    if re.match(r'.*\(колишній.*', cell):
        return cell.strip().split(' ')[0] +' район'
    else:
        return cell

def get_pop_dis(cell):
    if re.match(r'.*\sрайон', cell.strip()) or re.match(r'.*(міськрада).*', cell.strip()):
        newcell = re.sub(r'\(.*', '', cell)
        return newcell.strip()
    elif 'м. Інкерман' in cell:
        return 'Балаклавський'
    elif 'смт Кача' in cell:
        return 'Нахімовський'
    elif cell.strip() in wrong_towns:
        return cell.strip().split(' ')[1]
    elif re.match(r'\s\sм.*', cell):
        return cell.replace('м. ', '').strip()
    else:
        return np.nan
    
pop['town'] = pop['town'].apply(remove_ex)
pop['dis'] = pop['town'].apply(get_pop_dis)
pop['dis'] = pop['dis'].fillna(method='ffill')

In [17]:
def remove_dis(cell):
    if cell.strip().startswith('м.') or cell.strip().startswith('смт '):
        return cell.strip()
    else:
        return np.nan

pop['town'] = pop['town'].apply(remove_dis)
pop['town'] = pop['town'].str.replace(r'\(.*', '').str.strip()
pop['town'] = pop['town'].str.replace('м\.', 'м. ').str.replace('  ', ' ')
pop['dis'] = pop['dis'].str.replace('м\.', '').str.replace('  ', ' ')
pop['dis'] = pop['dis'].str.replace(r'\(.*', '').str.strip()
pop = pop.dropna(subset=['town'])

У цій таблиці не всі назви районів/обласних міст декомунізовані. Це можна виправити.

In [18]:
dekoa = koatuu_caps[koatuu_caps['koatuu'].str.endswith('00000')].copy()
dekoa = dekoa.dropna(subset=['oldname'])
dekdic = dict(zip(dekoa['oldname'], dekoa['town']))
dekobl = dict(zip(dekoa['oldname'], dekoa['obl']))

In [19]:
def decommunize(row):
    obl = row['obl']
    dis = row['dis']
    if dis.endswith('район'):
        return dis
    elif dis.upper() in dekdic.keys():
        if dekobl[dis.upper()] == obl:
            return remove_caps(dekdic[dis.upper()].title())
    return dis

pop['dis'] = pop.apply(decommunize, axis=1)
pop = pop.replace({'dis': disreplace}, regex=True)

In [20]:
#pop.to_csv('pop2016.csv', index=False)

In [21]:
for year in ['2014', '2015', '2016']:
    pop[year] = pd.to_numeric(pop[year], errors='coerce')

Тепер таблиці готові до об’єднання. Помилок лишитись не має.

In [22]:
newpop = pop.merge(koatuu, on=['town', 'obl', 'dis'], how='left')

In [23]:
newpop.to_csv('koatuu_towns_pop.csv', index=False)

Села найпростіше взяти з бази OpenStreetMap - судячи з їхніх форумів, вони серйозно намагались узяти найточніші дані, і у них є, окрім населення, ще й географічні координати.

OSM-дамп України узято звідси: http://download.geofabrik.de/europe.html

В архіві знаходиться .osm-файл у форматі xml, із якого витягнуто населені пункти за допомогою скрипту parse_osm.py (скрипт додається). Для скрипту використано бібліотеку osmread, яка погано працює з python 3 і тому її ключову фукнцію парсингу було дещо допиляно.

In [24]:
osm = pd.read_csv('ukr_towns.csv')

Кілька сіл таки відсутні в базі OSM, але створити невелику доповнювальну табличку вручну нескладно.

In [25]:
forgotten = pd.read_csv('forgotten.csv', dtype={'koatuu': str})
osm = pd.concat([osm, forgotten])

In [26]:
koatuu = koatuu[koatuu['np'] != 'Р']
newkoa = koatuu.merge(osm[['koatuu', 'lat', 'lon', 'population']], on='koatuu', how='left')

Понад 200 сіл не мають населення. Для них слід пропарсити сайт Верховної Ради.

In [27]:
import requests
from bs4 import BeautifulSoup
import time

table_url = 'http://w1.c1.rada.gov.ua/pls/z7503/a002'

In [28]:
r = requests.get(table_url)
soup = BeautifulSoup(r.text, 'lxml')

vrurl = 'http://w1.c1.rada.gov.ua/pls/z7503/'
oblinks = []
for link in soup.find_all('a'):
    if link.get('href') and ('SNP' in link.get('href')):
        oblinks.append(vrurl+link.get('href'))

In [29]:
dfs = []
count = 0
for oblink in oblinks:
    time.sleep(1)
    obr = requests.get(oblink)
    obsoup = BeautifulSoup(obr.text, 'lxml')
    vrobl = obsoup.findAll('h2')[0].text
    obl = re.sub("[\(\[].*?[\)\]]", "", vrobl)
    rows = []
    for tr in obsoup.findAll('tr')[1:]:
        tds = tr.findAll("td")
        selink = vrurl + tds[0].a['href']
        selo = (tds[0].text).strip()
        dis = (tds[1].text).strip().split(',')[0]
        vdic = {}
        vdic['town'] = selo
        vdic['link'] = selink
        vdic['dis'] = dis
        vdic['obl'] = obl
        dfs.append(pd.DataFrame(vdic, index=[count]))
        count += 1
villages = pd.concat(dfs)

In [30]:
villages['town'] = villages['town'].str.replace('c', 'с')
villages['town'] = villages['town'].str.replace(r'^с\s', 'с. ')
villages['town'] = villages['town'].str.replace(r'^с\-ще\s', 'с. ')
villages['town'] = villages['town'].str.replace(r"'", '’')
villages['town'] = villages['town'].str.replace(r"i", 'і')
villages['dis'] = villages['dis'].str.replace(r"'", '’')
villages['obl'] = villages['obl'].str.strip()
villages['dis'] = villages['dis'].str.strip()
villages['town'] = villages['town'].str.strip()

In [31]:
nopop = newkoa[newkoa['population'].isnull()]
vilnopop = nopop.merge(villages, how='left', on=['obl', 'dis', 'town'])

In [32]:
for i in vilnopop[vilnopop['link'].notnull()].index:
    link = vilnopop.loc[i]['link']
    r = requests.get(link)
    time.sleep(0.3)
    soup = BeautifulSoup(r.text, 'lxml')
    for tr in soup.findAll('tr'):
        tds = tr.findAll('td')
        if len(tds) < 3:
            continue
        if 'населення сільське' in tds[1].text.strip().lower():
            spop = tds[2].text
            if spop.strip() == '':
                spop = '0'
    vilnopop.loc[i, 'population'] = float(spop)*1000

In [33]:
newkoa2 = newkoa.merge(vilnopop[['koatuu', 'population']], how='left', on=['koatuu'])
newkoa2['population_x'] = newkoa2['population_x'].fillna(newkoa2['population_y'])
newkoa2 = newkoa2.drop('population_y', axis=1)
newkoa2 = newkoa2.rename(columns={'population_x': 'population'})

Кілька сіл доведеться проставити вручну (частина з них вже неясно чи існують, тому нулі)

In [34]:
# Біологічне - 0
# Новий Світ - 187
# Велика Яромірка - 1069
# Кукулівка - 82
# Нова Гута - 0
# Красна Гірка - 0
newkoa2.loc[newkoa2['koatuu'] == '1413591509', 'population'] = 187
newkoa2.loc[newkoa2['koatuu'] == '6321780508', 'population'] = 82
newkoa2.loc[newkoa2['koatuu'] == '6821281601', 'population'] = 1069
newkoa2['population'] = newkoa2['population'].fillna(0)

In [35]:
koatuu_pop = newkoa2.merge(newpop[['koatuu', '2014', '2015', '2016']], how='left', on='koatuu')

Тепер виставляємо найсвіжішу доступну інформацію про населення у відповідну колонку, і зберігаємо готовий датасет.

In [36]:
for i in koatuu_pop.index:
    if pd.notnull(koatuu_pop.loc[i, '2016']):
        koatuu_pop.loc[i, 'population'] = koatuu_pop.loc[i, '2016']
    elif pd.notnull(koatuu_pop.loc[i, '2015']):
        koatuu_pop.loc[i, 'population'] = koatuu_pop.loc[i, '2015']
    elif pd.notnull(koatuu_pop.loc[i, '2014']):
        koatuu_pop.loc[i, 'population'] = koatuu_pop.loc[i, '2014']
    else:
        continue

In [37]:
koatuu_pop = koatuu_pop.drop(['np', 'oblcode'], axis=1)
koatuu_pop = koatuu_pop.rename(columns = {'population': 'pop_latest', 
                                          '2014': 'pop_2014', '2015': 'pop_2015', '2016': 'pop_2016'})

koatuu_pop = koatuu_pop[['koatuu', 'town', 'type', 'obl', 'dis', 'oldname',
       'lat', 'lon', 'pop_2014', 'pop_2015', 'pop_2016', 'pop_latest']]

In [38]:
koatuu_pop = koatuu_pop.drop_duplicates(subset=['koatuu'])

In [39]:
koatuu_pop.to_csv('koatuu_pop.csv', index=False)

That's all, folks!

Звісно, бажано повторити всю процедуру через рік, коли будуть нові дані про населення міст, нова інфа у дампах OSM, уточнений класифікатор КОАТУУ, нові декомунізовані села і т.д.